<a href="https://colab.research.google.com/github/girishsenthil/Medical-Imaging-Models/blob/main/3DDoubleUNetKidneySegmentation(DSC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
% pip install jarvis-md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 2.8 MB/s 
     |████████████████████████████████| 596 kB 65.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import numpy as np, pandas as pd
import tensorflow as tf
from tensorflow.keras import Input, Model, models, layers, losses, metrics, optimizers
from jarvis.train import datasets

In [3]:
datasets.download(name='ct/kits')

[ 2022-07-18 15:56:50 ] [====================] 100.000% : Extracting archive (0000818 / 0000818) 

{'code': '/data/raw/ct_kits', 'data': '/data/raw/ct_kits'}

Implementing residual, inception, and squeeze&excite models

In [4]:
def residual(a, b):

  a_shape, b_shape = a.shape.as_list(), b.shape.as_list()

  if a_shape != b_shape:

    filters = a_shape[-1]
    i, j = b_shape[2] / a_shape[2], b_shape[3] / a_shape [3]

    strides = (1, int(i), int(j))

    kernel_size = 1 if strides == (1, 1, 1) else (1, 3, 3)

    b = layers.Conv3D(filters = filters, kernel_size = kernel_size,
                      strides = strides, padding = 'same')(b)
    return a + b

def inception(a, filters):

  # --- Define lambda functions
  conv = lambda x, filters,kernel_size : layers.Conv3D(
      filters=filters, 
      kernel_size=kernel_size, 
      padding='same')(x)

  norm = lambda x : layers.BatchNormalization()(x)
  relu = lambda x : layers.ReLU()(x)

  # --- Define 1x1, 3x3, 5x5 convs and 3x3 pools
  conv1 = lambda filters, x : relu(norm(conv(x, filters, kernel_size=(1, 1, 1))))
  conv3 = lambda filters, x : relu(norm(conv(x, filters, kernel_size=(1, 3, 3))))
  conv5 = lambda filters, x : relu(norm(conv(x, filters, kernel_size=(1, 5, 5))))
  mpool = lambda x : layers.MaxPool3D(pool_size=(1, 3, 3), strides=1, padding='same')(x)
  
  filters_sub = int(filters/4)

  p1 = conv1(filters_sub, a)
  p2 = conv3(filters_sub, conv1(filters, a))
  p3 = conv5(filters_sub, conv1(filters, a))
  p4 = conv1(filters_sub, mpool(a))

  return layers.Concatenate()([p1, p2, p3, p4])

def se(a, r=16):

  sqz = layers.AveragePooling3D((1, a.shape[2], a.shape[3]))(a)
  cha = int(a.shape[-1]/r)
  exc = layers.Conv3D(filters = cha, kernel_size = 1, activation = 'relu')(sqz)
  sca = layers.Conv3D(filters = a.shape[-1], kernel_size = 1, activation = 'sigmoid')(exc)

  return a * sca

In [5]:
def calculate_dsc(y_true, y_pred, c=1):
    """
    Method to calculate the Dice score coefficient for given class
    
    :params
    
      y_true : ground-truth label
      y_pred : predicted logits scores
           c : class to calculate DSC on
    
    """    
    true = y_true[..., 0] == c
    pred = tf.math.argmax(y_pred, axis=-1) == c 

    A = tf.math.count_nonzero(true & pred) * 2
    B = tf.math.count_nonzero(true) + tf.math.count_nonzero(pred)
    
    return tf.math.divide_no_nan(
        tf.cast(A, tf.float32), 
        tf.cast(B, tf.float32))

In [6]:
configs = {'batch': {'size': 2}}
gen_train, gen_valid, client = datasets.prepare(name='ct/kits', keyword='3d-bin', configs=configs, custom_layers=True)

In [7]:
kwargs = {
    'kernel_size': (3, 3, 3),
    'padding': 'same',
    'kernel_initializer': 'he_normal'}

# --- Define Building Block Lambda
conv = lambda x, filters, strides: layers.Conv3D(filters = filters, strides = strides,
                                                 **kwargs)(x)
tran = lambda x, filters, strides: layers.Conv3DTranspose(filters = filters, strides = strides,
                                                          **kwargs)(x)
norm = lambda x: layers.BatchNormalization()(x)
relu = lambda x: layers.ReLU()(x)

conv1 = lambda filters, x: relu(norm(conv(x, filters, strides = 1)))
conv2 = lambda filters, x: relu(norm(conv(x, filters, strides = (2, 2, 2))))
tran2 = lambda filters, x: relu(norm(tran(x, filters, strides = (2, 2, 2))))

concat = lambda a, b: layers.Concatenate()([a, b])

In [8]:
x = Input(shape=(96, 96, 96, 1), dtype='float32')

Upon learning about U-net, I wondered why this encoder-decoder model would not work better by just doubling the architecture and adding more skip connections in. However, as I have read more about different types of models I have come to the understanding that despite have a large amount of trainable parameters, they will not necessarily train well, or they will not yield significantly higher performance metrics for the trade-off of being extremely computationally intensive. 

Overall, I believe that smaller models with more effective mechanisms will out-perform this model

In [9]:
#Contract 1

l1 = conv1(8, x)
l2 = conv1(16, conv2(16, l1))
l3 = conv1(32, se(conv2(32, l2)))
l4 = conv1(48, se(conv2(48, l3)))
l5 = conv1(64, se(conv2(64, l4)))
l6 = conv1(80, se(conv2(80, l5)))

#Expand 1

l7 = tran2(64, l6)
l8 = tran2(48, conv1(64, l7 + l5))
l9 = tran2(32, conv1(48, l8 + l4))
l10 = tran2(16, conv1(32, l9 + l3))
l11 = tran2(8, conv1(16, l10 + l2))

#Contract 1

l12 = conv1(16, conv2(16, l11))
l13 = conv1(32, conv2(32, l12 + l10))
l14 = conv1(48, conv2(48, l13 + l9))
l15 = conv1(64, conv2(64, l14 + l8))
l16 = conv1(80, conv2(80, l15 + l7))

#Expand 1

l17 = tran2(64, conv1(80, l16 + l6))
l18 = tran2(48, conv1(64, l17 + l15))
l19 = tran2(32, conv1(48, l18 + l14))
l20 = tran2(16, conv1(32, l19 + l13))
l21 = tran2(8, conv1(16, l20 + l12))

In [10]:
logits = layers.Conv3D(filters = 2, **kwargs)

In [16]:
logits = {
    'c0': layers.Conv3D(filters=2, **kwargs)(l21),
    'c1': layers.Conv3D(filters=2, **kwargs)(l20),
}

## Define Model

In [17]:
backbone = Model(inputs = x, outputs = logits)

In [18]:
inputs = {
    'dat': Input(shape=(96, 96, 96, 1), name='dat'),
    'lbl': Input(shape=(96, 96, 96, 1), name='lbl')}

In [19]:
logits = backbone(inputs['dat'])

Adding Loss Parameters, Using SCE Loss to calculate pixel loss

In [20]:
loss = {}
true = inputs['lbl']

for c in sorted(logits.keys()):
  if c != 'c0':
    true = layers.MaxPooling3D(pool_size = (2, 2, 2))(true)
    
  loss[c] = losses.SparseCategoricalCrossentropy(from_logits = True, name = 'sce-' + c)(
      y_true = true,
      y_pred = logits[c])

In [21]:
dsc = calculate_dsc(y_true = inputs['lbl'], y_pred = logits['c0'])

In [22]:
training = Model(inputs = inputs, outputs = {**logits, **loss, **{'dsc': dsc}})

In [23]:
for loss in loss.values():
  training.add_loss(loss)

In [24]:
training.add_metric(dsc, name = 'dsc')

RMSprop loss with momentum works very well with deep learning models with large amounts of parameters

In [27]:
optimizer = optimizers.RMSprop(learning_rate = 3e-4)

In [26]:
client.load_data_in_memory()

[ 2022-07-18 16:10:35 ] [====================] 100.000% : Iterating | 000402    

In [29]:
training.compile(optimizer = optimizer)

In [30]:
training.fit(x = gen_train,
             steps_per_epoch = 100,
             epochs = 20,
             validation_data = gen_valid,
             validation_steps = 100,
             validation_freq = 5)

Epoch 1/20
100/100 [==============================] - 50s 270ms/step - loss: 0.6911 - dsc: 0.1075
Epoch 2/20
100/100 [==============================] - 28s 279ms/step - loss: 0.1262 - dsc: 0.6340
Epoch 3/20
100/100 [==============================] - 28s 285ms/step - loss: 0.0615 - dsc: 0.9245
Epoch 4/20
100/100 [==============================] - 28s 280ms/step - loss: 0.0493 - dsc: 0.9380
Epoch 5/20
100/100 [==============================] - 35s 353ms/step - loss: 0.0465 - dsc: 0.9423 - val_loss: 0.0448 - val_dsc: 0.9496
Epoch 6/20
100/100 [==============================] - 28s 282ms/step - loss: 0.0396 - dsc: 0.9495
Epoch 7/20
100/100 [==============================] - 28s 282ms/step - loss: 0.0318 - dsc: 0.9576
Epoch 8/20
100/100 [==============================] - 28s 282ms/step - loss: 0.0370 - dsc: 0.9533
Epoch 9/20
100/100 [==============================] - 28s 281ms/step - loss: 0.0278 - dsc: 0.9634
Epoch 10/20
100/100 [==============================] - 35s 346ms/step - loss: 0.0

In [35]:
from tensorflow import keras

In [ ]:
keras.utils.plot_model(backbone)